In [62]:
import numpy as np 
import pandas as pd 
import re

In [94]:
import cv2
import os

csv_file = "Licplatesdetection_train.csv"
df = pd.read_csv(csv_file)
df

,img_id,ymin,xmin,ymax,xmax
0,1.jpg,276,94,326,169
1,10.jpg,311,395,344,444
2,100.jpg,406,263,450,434
3,101.jpg,283,363,315,494
4,102.jpg,139,42,280,222
...,...,...,...,...,...
895,95.jpg,426,34,508,140
896,96.jpg,356,378,457,548
897,97.jpg,229,149,283,217
898,98.jpg,272,252,300,383


In [95]:
images_dir = 'D:\ObjectDetectionProject\dataset\images\license_plates_detection_train'
labels_dir = 'D:\ObjectDetectionProject\dataset\labels'
df.head()

<>:1: SyntaxWarning: invalid escape sequence '\O'
<>:2: SyntaxWarning: invalid escape sequence '\O'
<>:1: SyntaxWarning: invalid escape sequence '\O'
<>:2: SyntaxWarning: invalid escape sequence '\O'
C:\Users\Omer Siddiqui\AppData\Local\Temp\ipykernel_16376\4263091954.py:1: SyntaxWarning: invalid escape sequence '\O'
  images_dir = 'D:\ObjectDetectionProject\dataset\images\license_plates_detection_train'
C:\Users\Omer Siddiqui\AppData\Local\Temp\ipykernel_16376\4263091954.py:2: SyntaxWarning: invalid escape sequence '\O'
  labels_dir = 'D:\ObjectDetectionProject\dataset\labels'


,img_id,ymin,xmin,ymax,xmax
0,1.jpg,276,94,326,169
1,10.jpg,311,395,344,444
2,100.jpg,406,263,450,434
3,101.jpg,283,363,315,494
4,102.jpg,139,42,280,222


In [96]:
os.makedirs(labels_dir, exist_ok=True)

In [97]:
print('files in images directory:')
print(os.listdir(images_dir))

print('image id from csv')
print(df['img_id'].unique())


files in images directory:
['1.jpg', '10.jpg', '100.jpg', '101.jpg', '102.jpg', '103.jpg', '104.jpg', '105.jpg', '106.jpg', '107.jpg', '108.jpg', '109.jpg', '11.jpg', '110.jpg', '111.jpg', '112.jpg', '113.jpg', '114.jpg', '115.jpg', '116.jpg', '117.jpg', '118.jpg', '119.jpg', '12.jpg', '120.jpg', '121.jpg', '122.jpg', '123.jpg', '124.jpg', '125.jpg', '126.jpg', '127.jpg', '128.jpg', '129.jpg', '13.jpg', '130.jpg', '131.jpg', '132.jpg', '133.jpg', '134.jpg', '135.jpg', '136.jpg', '137.jpg', '138.jpg', '139.jpg', '14.jpg', '140.jpg', '141.jpg', '142.jpg', '143.jpg', '144.jpg', '145.jpg', '146.jpg', '147.jpg', '148.jpg', '149.jpg', '15.jpg', '150.jpg', '151.jpg', '152.jpg', '153.jpg', '154.jpg', '155.jpg', '156.jpg', '157.jpg', '158.jpg', '159.jpg', '16.jpg', '160.jpg', '161.jpg', '162.jpg', '163.jpg', '164.jpg', '165.jpg', '166.jpg', '167.jpg', '168.jpg', '169.jpg', '17.jpg', '170.jpg', '171.jpg', '172.jpg', '173.jpg', '174.jpg', '175.jpg', '176.jpg', '177.jpg', '178.jpg', '179.jpg', '18

In [98]:
def convert_to_yolo_format(row, image_width, image_height):
    xmin, ymin, xmax, ymax = row['xmin'], row['ymin'], row['xmax'], row['ymax']
    x_center = (xmin + xmax) /2 / image_width
    y_center = (ymin + ymax) /2 / image_height
    width = (xmax - xmin) / image_width
    height = (ymax - ymin) / image_height
    return x_center, y_center, width, height



In [101]:
for img_id in df['img_id'].unique():
    img_path = os.path.join(images_dir, img_id)
    if not os.path.isfile(img_path):
        print(f"warning , {img_path} not found")
        continue
    img = cv2.imread(img_path)
    if img is None:
        print("error loading image")
        continue
    img_height, img_width = img.shape[:2]
    annotations = df[df['img_id'] == img_id]
    yolo_annotations = []
    for _, row in annotations.iterrows():
        yolo_ann = convert_to_yolo_format(row, img_width, img_height)
        yolo_annotations.append(yolo_ann)
    
    txt_filename = img_id.replace('.jpg', '.txt')
    txt_path = os.path.join(labels_dir, txt_filename)

    with open(txt_path, 'w') as f:
        for ann in yolo_annotations:
            f.write(f"0 {ann[0]} {ann[1]} {ann[2]} {ann[3]}\n")